In [44]:
import time
import pandas as pd
import numpy as np
import requests, json

In [52]:
final_add = []
def get_coordinate(address): # 주소를 입력받음
    result = "" 
    address = str(address)
 
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address # 카카오 api 서버스를 이용하여 접근
    rest_api_key = '키' # 사용자 api key
    header = {'Authorization': 'KakaoAK ' + rest_api_key} 

    r = requests.get(url, headers=header) 
 
    if r.status_code == 200: # 정보를 오류없이 받아왔다면
        if len(r.json()['documents']) != 0: # 길이가 0이 아니라면
            try : 
                result_address = r.json()["documents"][0]["address"]
                result = (result_address["y"],result_address["x"])  # 좌표 정보에 접근
                final_add.append(result)
            except:  # 예외처리
                result = np.nan
            
        else: # 길이가 0인경우 na처리
            result = np.nan
            final_add.append(result)
    else:
        result = "ERROR[" + str(r.status_code) + "]"
    return result # 좌표를 반환

get_coordinate('경기 광명시 소하동 1340-3')

('37.448330720832', '126.882326488652')

In [46]:
#주소 검색 반복으로 지오코딩적용
df = pd.read_csv('전국 우체국 현황_20210630_장위우체국x.csv', encoding ='CP949')

In [47]:
df.head()

,지방우정청,총괄국명,우체국명,주소(지번),주소(도로명),전화번호
0,서울지방우정청,서울중앙우체국,서울중앙우체국,서울특별시 중구 충무로1가 21-1,서울특별시 중구 소공로 70 (층무로1가),02)6450-1114
1,서울지방우정청,서울중앙우체국,서울을지로4가우체국,서울특별시 중구 을지로4가 312-1,서울특별시 중구 을지로 154-2 (을지로4가),02)2273-6505
2,서울지방우정청,서울중앙우체국,서울태평로우체국,서울특별시 중구 봉래동1가 5-36 우리빌딩 2층,서울특별시 중구 칠패로 42 우리빌딩 2층 (봉래동1가),02)3783-7025
3,서울지방우정청,서울중앙우체국,서울퇴계로5가우체국,서울특별시 중구 장충동2가 173-3,서울특별시 중구 퇴계로 270 (장충동2가),02)2273-6508
4,서울지방우정청,서울중앙우체국,서울역전우체국,서울특별시 중구 봉래동2가 123,서울특별시 중구 통일로 21 (봉래동2가),02)757-1140


In [48]:
df.columns

Index(['지방우정청', '총괄국명', '우체국명', ' 주소(지번) ', ' 주소(도로명) ', ' 전화번호 '], dtype='object')

In [49]:
df.columns =['region_office', 'region_office_name', 'post_name', 'address_old', 'address', 'tel']

In [50]:
df.columns

Index(['region_office', 'region_office_name', 'post_name', 'address_old',
       'address', 'tel'],
      dtype='object')

In [53]:
xy = []
for address in df['address']:
    xy.append(get_coordinate(address))

df2 = pd.DataFrame(xy, columns =['lat','long'])

In [54]:
df2.head()
## 우체국이전 등은 제외하였음, 오전오후 바뀌는곳도 제외

,lat,long
0,37.5615987374058,126.982169090151
1,37.5662028147426,126.995163067598
2,37.5594743747373,126.973559809484
3,37.5627437974069,127.001652265803
4,37.5575861021915,126.971400388297


In [55]:
df2.shape

(3392, 2)

In [56]:
#df 병합
result = pd.concat([df,df2],axis=1)

In [58]:
result.head()

,region_office,region_office_name,post_name,address_old,address,tel,lat,long
0,서울지방우정청,서울중앙우체국,서울중앙우체국,서울특별시 중구 충무로1가 21-1,서울특별시 중구 소공로 70 (층무로1가),02)6450-1114,37.5615987374058,126.982169090151
1,서울지방우정청,서울중앙우체국,서울을지로4가우체국,서울특별시 중구 을지로4가 312-1,서울특별시 중구 을지로 154-2 (을지로4가),02)2273-6505,37.5662028147426,126.995163067598
2,서울지방우정청,서울중앙우체국,서울태평로우체국,서울특별시 중구 봉래동1가 5-36 우리빌딩 2층,서울특별시 중구 칠패로 42 우리빌딩 2층 (봉래동1가),02)3783-7025,37.5594743747373,126.973559809484
3,서울지방우정청,서울중앙우체국,서울퇴계로5가우체국,서울특별시 중구 장충동2가 173-3,서울특별시 중구 퇴계로 270 (장충동2가),02)2273-6508,37.5627437974069,127.001652265803
4,서울지방우정청,서울중앙우체국,서울역전우체국,서울특별시 중구 봉래동2가 123,서울특별시 중구 통일로 21 (봉래동2가),02)757-1140,37.5575861021915,126.971400388297


In [60]:
result.shape

(3392, 8)

In [61]:
result.to_csv('우체국_geo_이전_오전오후제거.csv',index = False, encoding ='euc-kr')